# Eco-inspired Indicators

# Instruction:

## 1. Network Construction in CSV File (Input-Output Table Format):

### Construct the network of flows between compartments in a CSV file in the format of an input-output table.
### Each supply chain actor or process should be represented as a node in the CSV.
### Ensure the CSV file is stored in a repository accessible via URL.
### The input-output numerical data should start from cell (2, 2) in the CSV and continue to the right and down, building the table (see one of the csv files for the multilayer film as an example)
### Node Order in CSV:
### The last four nodes are transactions with the external environment and must represent:
### - Fourth to last node : Virgin external resource node 
### - Third to last node : Renewable or recovered external resource node 
### - Second to last node : Dissipation node (for losses)
### - Last node: Export node (for open-loop recovery)

### Except for the last four nodes, other nodes can be in any order in the CSV file.

## 2. Running the Code:

### After constructing the network CSV file, execute the following code cell.

### The code will prompt you for several inputs:

### - URL of the CSV file containing the input-output flow network
### - Amount of refusal flow (in the same unit as the flows in the network)
### - Index of the consumer node (The indices start from 0 for the first column (or row) in the input-output table and continues incrementally as you move to the next column or row)
### - Indices of source nodes directing flows to the consumer node 
### - Indices of nodes in each circularity level (if applicable)
### - Indices of recovery processes (if applicable)
### - Indices of nodes receiving the recovered flows from recovery processes (if applcable)

### Provide accurate inputs as requested by the prompts.

In [43]:
#Importing packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.special import logsumexp

#CSV file URL
csv_url = input("Enter the URL of the CSV file (ending in .csv): ")

#Read CSV file, skip first two rows, and drop first two columns
df = pd.read_csv(csv_url, header=None, skiprows=2)
df = df.iloc[:, 2:]

# Building the total flow matrix
R = df.to_numpy()

# Refusal flow
F_R = float(input("Enter the amount of the refusal flow: "))



# Get the dimensions of the matrix
rows, cols = R.shape

# Constructing the Internal Flow Matrix (F)
# Specify how many rows and columns to remove
rows_to_remove = 4
cols_to_remove = 4
# Remove the last rows and columns
if rows_to_remove > 0 and cols_to_remove > 0:
    F = R[:rows - rows_to_remove, :cols - cols_to_remove]
    
# Consumer demand (functional unit)
consumer_node_index = int(input("Enter the index of the consumer node: "))
# Add all flows directing to the consumer node to calculate the functional unit
source_node_indices = input("Enter the indices of source nodes where the flow to the consumer node is coming from (comma-separated): ").split(",")
FU = sum(F[int(index), consumer_node_index] for index in source_node_indices)

# Virgin input vector (Z)
# Get the fourth row from the end and store it in a vector
Z = R[-4, :-4]

# Recovered input vector (Z_prime)
# Get the third row from the end and store it in a vector
Z_prime = R[-3, :-4]

# Dissipation vector (Y)
# Get the second last column and store it in a vector
Y = R[:, -2][:-4]

# Export vector (Y_prime)
# Get the last column and store it in a vector
Y_prime = R[:, -1][:-4]

# Number of compartments
n = len(F)

# Total boundary input
TBI = np.sum(Z) + np.sum(Z_prime)

# Total Input (Output) Throughflows
T_i = np.sum(F, axis=0) + Z + Z_prime

# Flow Analysis (identifies non-dimensional flow intensities along indirect pathways)
epsilon = 1e-10
G = F / (T_i + epsilon)

# Dimensionless Integral intensities matrix
I = np.identity(n, dtype='float64')
N = np.linalg.inv(I - G)

# Development capacity
DC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))

# Ascendency
ASC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = (np.sum(R[i, :]) + epsilon) * (np.sum(R[:, j]) + epsilon)
            ASC += numerator * np.log2((numerator * np.sum(R) / denominator) + epsilon)

# Redundancy
PHI = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = (np.sum(R[i, :]) + epsilon) * (np.sum(R[:, j]) + epsilon)
            PHI += -numerator * np.log2((numerator ** 2 / denominator) + epsilon)

# Ratio of ascendency to development capacity
R_ASC = ASC / DC

# Total System Throughflow
TST_flow = np.sum(F)

# Eco-inspired metrics

# Network Intensity
NI = TST_flow / FU

# Adjusted Network Intensity
NI_star = TST_flow / (FU + F_R)

# Robustness
ecological_resilience = -R_ASC * math.log(R_ASC)
Robustness = ecological_resilience / 0.3678

# Mean Circularity Level (MCL)
m = 8
Num = 0.0
Denom = 0.0

# Circularity Levels Vector
CLi = [1,2,3,4,5,6,7,8]

# Vector of flows in each circularity level
BLi = []


# Prompt user for indices of nodes in each circularity level with descriptions
circularity_levels_description = [
    "Circularity level 1 (Littering)",
    "Circularity level 2 (Landfill)",
    "Circularity level 3 (Energy Recovery, Compost)",
    "Circularity level 4 (Open-loop Recovery, Renewable)",
    "Circularity level 5 (Refurbish, Recycle)",
    "Circularity level 6 (Redistribute)",
    "Circularity level 7 (Reuse)"
]

BLi = []
for i in range(m - 1):
    indices = input(f"Enter the indices of nodes in {circularity_levels_description[i]} (comma-separated, leave empty if none): ").split(",")
    if indices == ['']:
        BLi.append(0)
    else:
        BLi.append(sum(T_i[int(index)] for index in indices))

BLi.append(F_R)

for i in range(m):
    Num += BLi[i] * CLi[i]
    Denom += BLi[i]
MCL = Num / Denom
MCL = (MCL - CLi[0]) / (CLi[-1] - 1)  # Normalizing

# Calculate R:V ratio
recovery_process_indices = input("Enter the indices of recovery processes (comma-separated, leave empty if there is none): ").split(",")
receiving_process_indices = input("Enter the indices of processes receiving the recovered flows in a closed-loop manner (comma-separated, leave empty if there is none): ").split(",")

if recovery_process_indices == ['']:
    numerator = 0
else:
    numerator = (
        sum(F[int(index1), int(index2)] for index1 in recovery_process_indices for index2 in receiving_process_indices) +
        sum(Y_prime[int(index)] for index in recovery_process_indices) +
        sum(Z_prime[int(index)] for index in recovery_process_indices)
    )

D_H = numerator / np.sum(Z)


R_V = numerator / np.sum(Z)

# Loop Tightness
FU_c = (N[consumer_node_index, consumer_node_index] - 1) / (N[consumer_node_index, consumer_node_index] + epsilon) * T_i[consumer_node_index]
LT = FU_c / FU

# Resource Consumption Efficiency
RCE = FU / (np.sum(Z) + epsilon)

# Adjusted Resource Consumption Efficiency
RCE_star = (FU + F_R) / (np.sum(Z) + epsilon)

# Print eco-inspired indicators
print("\033[1mEco-inspired Indicators:\033[0m")
print("Recovered-Renewable to Virgin ratio (R : V) =", R_V)
print("Resource Consumption Efficiency (RCE) =", RCE)
print("Adjusted Resource Consumption efficiency (RCE_star) =", RCE_star)
print("Network Intensity (NI) =", NI)
print("Adjusted Network Intensity (NI_star) =", NI_star)
print("Robustness =", Robustness)
print("Loop Tightness (LT) =", LT)
print("Mean Circularity Level (MCL) =", MCL)

Enter the URL of the CSV file (ending in .csv):  https://raw.githubusercontent.com/FarshidNazemi/ENA/main/data/Best%20Case%20(Refuse%2BDiverse%20EoL).csv
Enter the amount of the refusal flow:  0.8
Enter the index of the consumer node:  2
Enter the indices of source nodes where the flow to the consumer node is coming from (comma-separated):  1,4
Enter the indices of nodes in Circularity level 1 (Littering) (comma-separated, leave empty if none):  
Enter the indices of nodes in Circularity level 2 (Landfill) (comma-separated, leave empty if none):  7
Enter the indices of nodes in Circularity level 3 (Energy Recovery, Compost) (comma-separated, leave empty if none):  
Enter the indices of nodes in Circularity level 4 (Open-loop Recovery, Renewable) (comma-separated, leave empty if none):  6
Enter the indices of nodes in Circularity level 5 (Refurbish, Recycle) (comma-separated, leave empty if none):  5
Enter the indices of nodes in Circularity level 6 (Redistribute) (comma-separated, leav

Eco-inspired Indicators:
Recovered-Renewable to Virgin ratio (R : V) = 1.7204301075268817
Resource Consumption Efficiency (RCE) = 2.150537632096196
Adjusted Resource Consumption efficiency (RCE_star) = 10.75268816048098
Network Intensity (NI) = 3.4835000000000003
Adjusted Network Intensity (NI_star) = 0.6967000000000001
Robustness = 0.7059529566199914
Loop Tightness (LT) = 0.5804511262775637
Mean Circularity Level (MCL) = 0.9076465157298571
